* The big problem with the @property built-in is reuse. 
* The methods it decorates can’t be reused for multiple attributes of the same class. They also can’t be reused by unrelated classes.
For example, say you want a class to validate that the grade received by a student

In [1]:
# Example 1
class Homework(object):
    def __init__(self):
        self._grade = 0

    @property
    def grade(self):
        return self._grade

    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._grade = value


# Example 2
galileo = Homework()
galileo.grade = 95
print(galileo.grade)



95


* 

In [2]:
# Example 3
class Exam(object):
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0

    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')


# Example 4
    @property
    def writing_grade(self):
        return self._writing_grade

    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value

    @property
    def math_grade(self):
        return self._math_grade

    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value


# Example 5
galileo = Exam()
galileo.writing_grade = 85
galileo.math_grade = 99
print('Writing: %5r' % galileo.writing_grade)
print('Math:    %5r' % galileo.math_grade)




Writing:    85
Math:       99


* If you want to reuse.
* The better way to do this in Python is to use a descriptor. 
* The descriptor protocol defines how attribute access is interpreted by the language. 
* A descriptor class can provide __get__ and __set__ methods that let you reuse the grade validation behavior without any boilerplate

For this purpose, descriptors are also better than mix-ins (see Item 26:
“Use Multiple Inheritance Only for Mix-in Utility Classes”) because they let you reuse the
same logic for many different attributes in a single class.

* The Grade class implements the descriptor protocol.

In [3]:
# Example 6
class Grade(object):
    def __get__(*args, **kwargs):
        pass

    def __set__(*args, **kwargs):
        pass

class Exam(object):
    # Class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    
# Example 7
exam = Exam()
exam.writing_grade = 40


# Example 8
Exam.__dict__['writing_grade'].__set__(exam, 40)


# Example 9
print(exam.writing_grade)


# Example 10
print(Exam.__dict__['writing_grade'].__get__(exam, Exam))
    

None
None


* What drives this behavior is the __getattribute__ method of object 
 (see Item 32: “Use __getattr__, __getattribute__, and __setattr__ for Lazy Attributes”). 
* In short, when an Exam instance doesn’t have an attribute named writing_grade, Python will fall back to the Exam class’s attribute instead. If this class attribute is an object that has __get__ and __set__ methods, Python will assume you want to follow the descriptor protocol

* Knowing this behavior and how I used @property for grade validation in the Homework class, here’s a reasonable first attempt at implementing the Grade descriptor.


In [4]:
# Example 11
class Grade(object):
    def __init__(self):
        self._value = 0

    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value

class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()


# Example 12
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)


# Example 13
second_exam = Exam()
second_exam.writing_grade = 75
print('Second', second_exam.writing_grade, 'is right')
print('First ', first_exam.writing_grade, 'is wrong') # <<<<< Problem

Writing 82
Science 99
Second 75 is right
First  75 is wrong


[Problem - upper side code]
The problem is that a single Grade instance is shared across all Exam instances for the
class attribute writing_grade. The Grade instance for this attribute is constructed
once in the program lifetime when the Exam class is first defined, not each time an Exam
instance is created.


>>> To solve this, 
I need the Grade class to keep track of its value for each unique Exam
instance. I can do this by saving the per-instance state in a dictionary.

This implementation is simple and works well, but there’s still one gotcha: It leaks
memory. The _values dictionary will hold a reference to every instance of Exam ever
passed to __set__ over the lifetime of the program. This causes instances to never have
their reference count go to zero, preventing cleanup by the garbage collector.

>>> To fix this, 
I can use Python’s weakref built-in module. This module provides a special
class called WeakKeyDictionary that can take the place of the simple dictionary used
for _values. The unique behavior of WeakKeyDictionary is that it will remove
Exam instances from its set of keys when the runtime knows it’s holding the instance’s
last remaining reference in the program. Python will do the bookkeeping for you and
ensure that the _values dictionary will be empty when all Exam instances are no longer
in use.

In [5]:
# Example 14
class Grade(object):
    def __init__(self):
        self._values = {}

    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value


# Example 15
from weakref import WeakKeyDictionary

class Grade(object):
    def __init__(self):
        self._values = WeakKeyDictionary()
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value


# Example 16
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print('First ', first_exam.writing_grade, 'is right')
print('Second', second_exam.writing_grade, 'is right')

First  82 is right
Second 75 is right


* Reuse the behavior and validation of @property methods by defining your own descriptor classes.
* Use WeakKeyDictionary to ensure that your descriptor classes don’t cause memory leaks.
* Don’t get bogged down trying to understand exactly how __getattribute__ uses the descriptor protocol for getting and setting attributes.